In [1]:
import polars as pl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import mappings

# Set the theme once
pio.templates.default = "plotly_white"

from util import multi_select_counts, get_subplot_coords

df_pre = pl.read_csv("OriginalData/Preprogram_Survey.csv")
df_post = pl.read_csv("OriginalData/PostOrientation_Survey.csv")

In [2]:
df_growth = df_post.select(["#"] + mappings.growth_questions).to_pandas()

In [3]:
df_growth = pl.DataFrame(df_growth.replace(mappings.all_post_orientation_entry_mappings).rename(columns=mappings.original_to_concise_question_names))

# Define the logical order for each set of responses
# These lists should contain the *standardized* entries in your desired order
order_confidence_emotions = ['Highly Confident', 'Very Confident', 'Slightly Confident', 'Not Confident At All']
order_emotional_triggers = ['Significantly Better', 'Much Better', 'Slightly Better', 'Not Better At All']
order_communication_style = ['Clearly Understand', 'Understand & Improving', 'General Understanding', 'Do Not Understand']
order_goal_motivation = ['Highly Motivated', 'Much More Motivated', 'Slightly More Motivated', 'No Change', 'Less Motivated']
order_career_paths = ['Very Clear', 'Solid Ideas', 'Slightly Clearer', 'Still Unsure']
order_money_management = ['Highly Confident', 'Very Confident', 'Somewhat Confident', 'Not Confident At All'] # Duplicate of confidence, but good to keep explicit
order_professional_presence = ['Highly Confident', 'Very Confident', 'Somewhat Confident', 'Slightly More Confident', 'Not Yet Confident']
order_overall_growth = ['More Than Expected', 'Significant Growth', 'Some Growth', 'Slight Growth', 'No Growth At All']

# A dictionary to map question names (or a unique identifier for them) to their respective orders
post_orientation_question_orders = {
    "Confidence in Managing Emotions": order_confidence_emotions,
    "Recognizing Emotional Triggers": order_emotional_triggers,
    "Understanding Communication Style": order_communication_style,
    "Motivation for Goal Setting": order_goal_motivation,
    "Clarity on Career Paths": order_career_paths,
    "Confidence in Money Management": order_money_management,
    "Confidence in Professional Presence": order_professional_presence,
    "Overall Growth": order_overall_growth
}

In [4]:
fig = make_subplots(rows=2, cols=4,
                    subplot_titles=list(post_orientation_question_orders.keys()))

for index, g in enumerate(df_growth.columns[1:]):
    
    position = get_subplot_coords(index, 2, 4)
    
    d = multi_select_counts(df_growth, ["#", g])
    fig.add_trace(
        go.Bar(y=d.values, x=d.index),
        row=position[0], col=position[1])
    
    fig.update_xaxes(
        categoryorder='array', # Ensure Plotly respects the order of the array
        categoryarray=post_orientation_question_orders[g], # Pass the order directly to Plotly for good measure
        tickangle=-45, # Adjust as needed for label readability
        row=position[0], col=position[1]
    )

In [5]:
# Update layout
fig.update_layout(height=1000, width=1800, title_text="")
fig.update_layout(showlegend=False)
"""fig.update_xaxes(
    categoryorder='array',
    categoryarray=['Not at all', 'Slightly', 'Moderately','Very', 'Extremely'])"""
fig.show()

In [6]:
post_orientation_mapping = {
    "How confident are you now in managing your emotions and staying calm when things get stressful at work or in life?": [
        "I can manage challenges without giving up"
    ],
    "After learning about emotional intelligence, how are you at recognizing your personal emotional triggers?": [
        "Stressed or overwhelmed"
    ],
    "After the Communication Styles workshop, how well do you understand your own communication style?": [
        "Communicating professionally (verbal and written)"
    ],
    "Are you more or less motivated about setting and actually reaching your goals after the goals workshop?": [
        "Motivated to work toward your goals",
        "Focused and productive"
    ],
    "What is your current clarity level about your possible career paths?": [
        "Do you have a *career plan or clear next steps?*"
    ],
    "How confident are you in your ability *to manage your money* this summer?": [
        "Creating a personal budget",
        "Understanding credit and credit scores",
    ],
    "After the personal branding session, how confident are you in *how you show up professionally*?": [
        "I know how to present myself professionally in different settings"
    ],
    "Overall, how much do you feel you grew during the orientation?": []
}

all_matched_columns = [col for cols in post_orientation_mapping.values() for col in cols]


In [7]:
df_pre.select(all_matched_columns)

I can manage challenges without giving up,Stressed or overwhelmed,Communicating professionally (verbal and written),Motivated to work toward your goals,Focused and productive,Do you have a *career plan or clear next steps?*,Creating a personal budget,Understanding credit and credit scores,I know how to present myself professionally in different settings
str,str,str,str,str,str,str,str,str
"""Agree""","""Very Often""","""Very""","""Often""","""Often""","""I have a general idea but need…","""Extremely""","""Somewhat""","""Agree"""
"""Agree""","""Sometimes""","""Slightly ""","""Very Often""","""Sometimes""","""Yes, I have a clear plan I’m w…","""Very""","""Somewhat""","""Neutral """
"""Strongly agree""","""Sometimes""","""Extremely""","""Very Often""","""Very Often""","""I’m still exploring and need g…","""Very""","""Very""","""Strongly agree"""
"""Strongly agree""","""Rarely""","""Somewhat""","""Very Often""","""Often""","""I have a general idea but need…","""Very""","""Somewhat""","""Strongly agree"""
"""Neutral ""","""Very Often""","""Somewhat""","""Very Often""","""Sometimes""","""I’m still exploring and need g…","""Somewhat""","""Somewhat""","""Agree"""
…,…,…,…,…,…,…,…,…
"""Strongly disagree""","""Rarely""","""Not confident""","""Sometimes""","""Sometimes""","""I’m still exploring and need g…","""Slightly""","""Not familiar""","""Strongly disagree"""
"""Neutral ""","""Sometimes""","""Somewhat""","""Often""","""Often""","""I have a general idea but need…","""Somewhat""","""Somewhat""","""Agree"""
"""Neutral ""","""Sometimes""","""Somewhat""","""Often""","""Often""","""I have a general idea but need…","""Somewhat""","""Extremely""","""Neutral """


In [8]:
all_matched_columns

['I can manage challenges without giving up',
 'Stressed or overwhelmed',
 'Communicating professionally (verbal and written)',
 'Motivated to work toward your goals',
 'Focused and productive',
 'Do you have a *career plan or clear next steps?*',
 'Creating a personal budget',
 'Understanding credit and credit scores',
 'I know how to present myself professionally in different settings']